# Bereinigung der Datensätze

## 1. Importieren der Pakete

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf 
import seaborn as sns


## 2. Kundendaten

### 2.1 Kundendatei einlesen

In [ ]:
kunden = pd.read_csv("kunden.csv", decimal = '.', sep = ";")
kunden.head()

### 2.2 Überblick über Datentypen

In [ ]:
kunden.dtypes

### 2.3. Bereinigung

### 2.3.1 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
kunden.isnull().sum()

In [ ]:
# Finden des Modalwertes:

In [ ]:
modalwert_kunden = kunden['Geschlecht'].mode()[0]

In [ ]:
modalwert_kunden

In [ ]:
# Ersetzen der 5 fehlenden Geschlechtseinträge mit dem Modalwert 1.0

In [ ]:
kunden.fillna(value = modalwert_kunden, axis = 0, inplace = True)

In [ ]:
# Überprüfung
kunden.isnull().sum()

### 2.3.2 Ausreißer ermitteln und entfernen

In [ ]:
# Außreißer ermitteln
kunden.describe().T

### 2.3.2.1 Alter 

In [ ]:
boxplot_alter = kunden.boxplot(column = 'Alter')
plt.show()

In [ ]:
# zwei Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR
Q1_Alter = kunden['Alter'].quantile(0.25)
Q3_Alter = kunden['Alter'].quantile(0.75)
IQR_Alter = Q3_Alter - Q1_Alter
# Berechnung der Ausreißer-Grenzen
untere_grenze_Alter = Q1_Alter - 1.5 * IQR_Alter
obere_grenze_Alter = Q3_Alter + 1.5 * IQR_Alter
kunden['Ausreißer'] = (kunden['Alter'] < untere_grenze_Alter) | (kunden['Alter'] > obere_grenze_Alter)
ausreißer_alter = kunden.Ausreißer.value_counts()
ausreißer_alter

In [ ]:
# Ermitteln der genauen Werte der Ausreißer

In [ ]:
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Personen im Alter von 100 und 120 Jahren werden kein Auto mehr kaufen

In [ ]:
# Ersetzen der Ausreißer in der Alters-Spalte durch Medianwert
alter_median = kunden['Alter'].median()
kunden.replace([120, 100], alter_median, inplace=True)

In [ ]:
# Überprüfen ob noch Ausreißer vorhanden sind
kunden['Ausreißer'] = (kunden['Alter'] < untere_grenze_Alter) | (kunden['Alter'] > obere_grenze_Alter)
ausreißer_alter = kunden.Ausreißer.value_counts()
ausreißer_alter

In [ ]:
# keine Ausreißer mehr vorhanden

## 2.3.2.2 Einkommen 

In [ ]:
boxplot_einkommen = kunden.boxplot(column = 'Einkommen')
plt.show()

In [ ]:
# 4 Ausreißer (aufgrund der Skalierung schwer zu erkennen)

In [ ]:
# Berechnung von Q1, Q3 und IQR der Einkommens-Spalte
Q1_einkommen = kunden['Einkommen'].quantile(0.25)
Q3_einkommen = kunden['Einkommen'].quantile(0.75)
IQR_einkommen = Q3_einkommen - Q1_einkommen
IQR_einkommen
untere_grenze_einkommen = Q1_einkommen - 1.5 * IQR_einkommen
obere_grenze_einkommen = Q3_einkommen + 1.5 * IQR_einkommen
kunden['Ausreißer'] = (kunden['Einkommen'] < untere_grenze_einkommen) | (kunden['Einkommen'] > obere_grenze_einkommen)
ausreißer_einkommen = kunden.Ausreißer.value_counts()
ausreißer_einkommen

In [ ]:
# Ermitteln der genauen Werte der 4 Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Ersetzen der Ausreißer in der Einkommens-Spalte durch Medianwert
einkommen_median = kunden['Einkommen'].median()
kunden.replace({'Einkommen': [1.000000e+09, 1.076890e+05, 0.000000e+00, -62840.374523]}, einkommen_median, inplace=True)

In [ ]:
# Überprüfen ob noch Ausreißer vorhanden sind
kunden['Ausreißer'] = (kunden['Einkommen'] < untere_grenze_einkommen) | (kunden['Einkommen'] > obere_grenze_einkommen)
ausreißer_einkommen = kunden.Ausreißer.value_counts()
ausreißer_einkommen

In [ ]:
# keine Ausreißer mehr vorhanden

### 2.3.2.3 Preis

In [ ]:
boxplot_preis = kunden.boxplot(column = 'Preis')
plt.show()

In [ ]:
# ein Ausreißer zu erkennen

In [ ]:
Q1_preis = kunden['Preis'].quantile(0.25)
Q3_preis = kunden['Preis'].quantile(0.75)
IQR_preis = Q3_preis - Q1_preis
IQR_preis
untere_grenze_preis = Q1_preis - 1.5 * IQR_preis
obere_grenze_preis = Q3_preis + 1.5 * IQR_preis
kunden['Ausreißer'] = (kunden['Preis'] < untere_grenze_preis) | (kunden['Preis'] > obere_grenze_preis)
ausreißer_preis = kunden.Ausreißer.value_counts()
ausreißer_preis

In [ ]:
# Ermitteln der genauen Werte der Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# bei einem Einkommen von 59.013 ist es überraschend sich ein Auto im Preis von 165.482,31 zu kaufen, aber möglich, daher wird der Wert behalten


### 2.3.2.4 Zeit

In [ ]:
boxplot_preis = kunden.boxplot(column = 'Zeit')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_zeit = kunden['Zeit'].quantile(0.25)
Q3_zeit = kunden['Zeit'].quantile(0.75)
IQR_zeit = Q3_zeit - Q1_zeit
IQR_zeit
untere_grenze_zeit = Q1_zeit - 1.5 * IQR_zeit
obere_grenze_zeit = Q3_zeit + 1.5 * IQR_zeit
kunden['Ausreißer'] = (kunden['Zeit'] < untere_grenze_zeit) | (kunden['Zeit'] > obere_grenze_zeit)
ausreißer_zeit = kunden.Ausreißer.value_counts()
ausreißer_zeit

In [ ]:
# Ermittlung der genauen Werte der Ausreißer
kunden.loc[kunden['Ausreißer'] == True]

In [ ]:
# Die Zeiten sind möglich, z.B. 12 Minuten weil keine Probefahrt gemacht wurde oder 80 Minuten weil eine lange gemacht wurde, Werte bleiben erhalten

In [ ]:
kunden

In [ ]:
# Boolean-Spalte wieder entfernen
kunden = kunden.drop(kunden.columns[-1], axis=1)
kunden

## 3. Besucherdaten

### 3.1 Besucherdatei einlesen

In [ ]:
besucher = pd.read_csv("besucher.csv", decimal = '.', sep = ";")
besucher.head()

### 3.2 Überblick über Datentypen

In [ ]:
besucher.dtypes

In [ ]:
# zum Vergleich: Kunden
kunden.dtypes

In [ ]:
# Alter = int, Einkommen = float (ändern bei besucher), Preis = float, Geschlecht = int (ändern bei kunden), Zeit = float (ändern bei besucher), KundeNr = object

### 3.3. Bereinigung

In [ ]:
# vermutlich ist die Kommaschreibweise aus dem deutschen, während wir die Werte in der englischen Schreibweise mit einem Punkt, statt Komma benötigen

### 3.3.1 Bereinigung der Nachkommastellen und Umwandlung der dtypes

### 3.3.1.1 Einkommen

In [ ]:
besucher["Einkommen"] = besucher["Einkommen"].str.replace(",",".")
besucher["Einkommen"] = pd.to_numeric(besucher["Einkommen"]) 
besucher["Einkommen"] = besucher["Einkommen"].round(0).astype("float64")
besucher

### 3.3.1.2 Zeit

In [ ]:
besucher["Zeit"] = besucher["Zeit"].str.replace(",",".")
besucher["Zeit"] = pd.to_numeric(besucher["Zeit"])
besucher["Zeit"] = besucher["Zeit"].round(0).astype("float64")
besucher

### 3.3.1.3 Umwandlung von Geschlecht von float in integer


In [ ]:
kunden["Geschlecht"] = kunden["Geschlecht"].astype("int64")
kunden

In [ ]:
# Überprüfen der dtypes
besucher.dtypes, kunden.dtypes

### 3.3.2 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
besucher.isnull().sum()

In [ ]:
# keine fehlenden Datenpunkte im Datensatz

### 3.3.3 Ausreißer ermitteln und entfernen

In [ ]:
# Außreißer ermitteln
kunden.describe().T

### 3.3.3.1 Alter

In [ ]:
boxplot_besucher_alter = besucher.boxplot(column = 'Alter')
plt.show()

In [ ]:
# sowohl bei describe-Tabelle als auch beim boxplot keine Ausreißer zu erkennen

### 3.3.3.2 Einkommen

In [ ]:
boxplot_besucher_einkommen = besucher.boxplot(column = 'Einkommen')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_einkommen_besucher = besucher['Einkommen'].quantile(0.25)
Q3_einkommen_besucher = besucher['Einkommen'].quantile(0.75)
IQR_einkommen_besucher = Q3_einkommen_besucher - Q1_einkommen_besucher
untere_grenze_einkommen_besucher = Q1_einkommen_besucher - 1.5 * IQR_einkommen_besucher
obere_grenze_einkommen_besucher = Q3_einkommen_besucher + 1.5 * IQR_einkommen_besucher
besucher['Ausreißer'] = (besucher['Einkommen'] < untere_grenze_einkommen_besucher) | (besucher['Einkommen'] > obere_grenze_einkommen_besucher)
ausreißer_einkommen_besucher = besucher.Ausreißer.value_counts()
ausreißer_einkommen_besucher

In [ ]:
# Ermitteln der genauen Werte der Ausreißer
besucher.loc[besucher['Ausreißer'] == True]

In [ ]:
# so niedriges und so hohes Einkommen ist möglich, daher bleiben Werte erhalten

### 3.3.3.3 Zeit

In [ ]:
boxplot_besucher_zeit = besucher.boxplot(column = 'Zeit')
plt.show()

In [ ]:
# mehrere Ausreißer zu erkennen

In [ ]:
# Berechnung von Q1, Q3 und IQR der Zeit-Spalte
Q1_zeit_besucher = besucher['Zeit'].quantile(0.25)
Q3_zeit_besucher = besucher['Zeit'].quantile(0.75)
IQR_zeit_besucher = Q3_zeit_besucher - Q1_zeit_besucher
untere_grenze_zeit_besucher = Q1_zeit_besucher - 1.5 * IQR_zeit_besucher
obere_grenze_zeit_besucher = Q3_zeit_besucher + 1.5 * IQR_zeit_besucher
besucher['Ausreißer'] = (besucher['Zeit'] < untere_grenze_zeit_besucher) | (besucher['Zeit'] > obere_grenze_zeit_besucher)
ausreißer_zeit_besucher = besucher.Ausreißer.value_counts()
ausreißer_zeit_besucher

In [ ]:
# Ermittlung der genauen Werte der Ausreißer
besucher.loc[besucher['Ausreißer'] == True]

In [ ]:
# so kurze und lange Zeiten sind möglich, daher bleiben Werte erhalten

In [ ]:
besucher

In [ ]:
# Boolean-Spalte wieder entfernen

In [ ]:
besucher = besucher.drop(besucher.columns[-1], axis=1)
besucher

# 4. Geo-Daten

### 4.1. Geodatei einlesen

In [ ]:
geo = pd.read_csv("geo.txt", sep = "\t")
geo.head()

## 4.2 Überblick über Datentypen

In [ ]:
geo.dtypes

In [ ]:
# bei kunden und besucher ist der dypte der KundeNr auch object, daher keine Änderung benötigt

### 4.3 Bereinigung

### 4.3.1 Fehlende Datenpunkte ermitteln und ersetzen

In [ ]:
geo.isnull().sum()

### 4.3.2 Ausreißer ermitteln und entfernen/ersetzen

In [ ]:
# Übersicht aller Niederlassungen
geo_niederlassung_uniques = geo["Niederlassung"].unique()
print(geo_niederlassung_uniques)

In [ ]:
# Bereinigung der Datenpunkte (NRW, Düsseldorf zu Nordrhein-Westfalen), (Berlin-Charlottenburg, Berlin-Mitte, BERLIN zu Berlin)
geo_cleaning = {
    "NRW" : "Nordrhein-Westfalen" , 
    "Berlin-Charlottenburg" : "Berlin" ,
    "Berlin-Mitte" : "Berlin" ,
    "BERLIN" : "Berlin",
    "Düsseldorf" : "Nordrhein-Westfalen"}

geo["Niederlassung"] = geo["Niederlassung"].replace(geo_cleaning, regex = True)

In [ ]:
# Überprüfung
geo_niederlassung_uniques2 = geo["Niederlassung"].unique()
print(geo_niederlassung_uniques2)

## 5. Zusammenführung der Listen

### 5.1 Kunden und Besucher mit Geo

### 5.1.1 Kunden mit Geo 

In [ ]:
kunden_geo = pd.merge(kunden,geo, on = "KundeNr")
kunden_geo

### 5.1.2 Besucher mit Geo

In [ ]:
besucher_geo = pd.merge(besucher, geo, on = "KundeNr")
besucher_geo

### 5.2 kunden_geo und besucher_geo zu df_final

In [ ]:
df_final = pd.concat([kunden_geo, besucher_geo], axis = 0, ignore_index = True)
df_final

### 5.3 CSV Datei df_final

In [ ]:
df_final.to_csv('df_final.csv', index=False)

In [ ]:
<html>
<body>
</body>
</html>

<html>
<body>
<font size = "8">Elektorauto AG</font><br>
<i>HWR Berlin, Sommersemester 2024 </i><br>
<i> Maximilian Czapons, Monika Nekipelova, Nicola Reitz</i><br>
<i> 27. Juni 2024 </i>
</body>
</html>

<font size = "8">CEO Pr&auml;sentation: Performance Analyse</font>

<html>
<body>
<font size = "8">Gliederung</font><br>
    <ol>
        <li> Verkaufszahlen </li>
        <li> Umsatzzahlen </li>
        <li> Regionale Verteilung </li>
        <li> Kundenanalyse </li>
        <li> Einfluss der sozio&ouml;konomischen Faktoren </li>
    </ol>
</body>
</html>

# Verkaufszahlen

<html>
<body>
<font size = "6"> F1: Wie viele Autos wurden verkauft</font>
    <h3>Trotz wenigen Monaten im Markt starke Zahlen:</h3>
        <ul> 
            <li> 1104 verkaufte Autos </li>
            <li> Eintritt gelungen</li>
        </ul>
</body>
</html>

<html>
<body>
<font size = "6">F2: Was ist der H&ouml;chst-, Mindest- und Durchschnittspreis?</font>
    <br><br><br>
<table border="1" class="dataframe" style="float: left;">
  <thead>
    <tr style="text-align: left;">
      <th></th>
      <th>Maximaler Verkaufspreis</th>
      <th>Minimaler Verkaufspreis</th>
      <th>Durchschnittspreis</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th></th>
      <td>165.482,31 Euro</td>
      <td>21.471,65 Euro</td>
      <td>88.975,20 Euro</td>
    </tr>
  </tbody>
</table>
</body>
</html>

<html>
<body>
    <h3> Gro&szlig;e Spanne </h3>
        <ul> 
            <li> Diversifizierte Produktlinie </li>
            <li> Diverse Zusatzoptionen mit Aufpreisen </li>
        </ul>

</body>
</html>

<html>
<body>
    <h3> Nutzen der Spanne </h3>
        <ul> Verschiedene Kostenoptionen (mit je Zusatzoptionen)
            <li> g&uuml;nstig </li>
            <li> mittel</li>
            <li> teuer</li>
        </ul>
        <ul> Verschiedene Modelle (mit je Zusatzoptionen)
            <li> Kleinwagen </li>
            <li> SUV</li>
            <li> Sportwagen </li>
        </ul>
</body>
</html>

# Umsatzzahlen


<html>
<body>
<font size = "6">F3: Wie hoch war der Gesamtumsatz?</font>
    <h3> In den wenigen Monaten im Markt beeindruckenden Umsatz erzielt</h3>
    <ul> 
        <li> Gesamtumsatz: 92.228.619,38 Euro</li>
        <li> Eintritt gelungen </li>
    </ul>
</body>
</html>

# Regionale Aufteilung

<html>    
<body>
    <font size="6">F4: Wie viele Autos wurden pro Bundesland verkauft?</font>
</body>
</html>

In [ ]:
autos_pro_bundesland = kunden_geo["Niederlassung"].value_counts()
autos_pro_bundesland


In [ ]:
# Erstellen eines Diagramms zur Darstellung
# Dataframe
daten = {"Bundesland" : ["Baden-Württemberg" , "Bayern" , "Nordrhein-Westfalen" , "Berlin" , "Hessen" , "Niedersachsen",
                        "Sachsen" , "Thüringen" , "Hamburg" , "Brandenburg"] , 
        "Verkäufe" : [205 , 188 , 175 , 126 , 96 , 96 , 89 , 49 , 43 , 37]}

df_autos_pro_bundesland = pd.DataFrame(daten)

verkäufe_durchschnitt = df_autos_pro_bundesland["Verkäufe"].mean()

# Diagramm
plt.figure(figsize=(10, 6))
diagramm = plt.bar(df_autos_pro_bundesland["Bundesland"],df_autos_pro_bundesland["Verkäufe"], color="lightsteelblue")
plt.axhline(y=verkäufe_durchschnitt, color='red', linestyle='--', linewidth=2)

for bar in diagramm:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{int(height)}", ha = "center", va = "center_baseline", fontsize=10)
    
plt.title("Anzahl der verkauften Autos pro Bundesland")
plt.xlabel("Bundesland")
plt.ylabel("Anzahl der verkauften Autos")
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()
plt.show()

<body>
<html>
<font size = "6">F5: Wie hoch war der durchschnittliche Umsatz pro Bundesland?</font>
</body>
</html>

In [ ]:
umsatz_pro_bundesland = kunden_geo.groupby("Niederlassung")["Preis"].agg("mean")
umsatz_pro_bundesland = umsatz_pro_bundesland.round(2)
umsatz_pro_bundesland

In [ ]:
# Erstellen eines Diagramms zur Darstellung
# Dataframe
daten2 = {"Bundesland" : ["Baden-Württemberg" , "Bayern" , "Nordrhein-Westfalen" , "Berlin" , "Hessen" , "Niedersachsen",
                        "Sachsen" , "Thüringen" , "Hamburg" , "Brandenburg"] , 
        "Umsatz" : [90969.56 , 89344.31 , 89584.98 , 86673.64 , 85549.50 , 85864.93 , 87963.31 , 88633.27 , 89455.08 , 90816.99]}

df_umsatz_pro_bundesland = pd.DataFrame(daten2)

# Diagramm
plt.figure(figsize=(10, 6))
diagramm2 = plt.bar(df_umsatz_pro_bundesland["Bundesland"],df_umsatz_pro_bundesland["Umsatz"], color="lightsteelblue")

for bar in diagramm2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{int(height)}", ha = "center", va = "center_baseline", fontsize=10)

plt.title("Umsatz pro Bundesland")
plt.xlabel("Bundesland")
plt.ylabel("Umsatz (in €)")
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()
plt.show()


<html>
<body>
<font size = "6"> F6: In welchem Bundesland wurde das teuerste Auto verkauft?</font>
    <h3>Bundesland und Preis:</h3>
        <ul> 
            <li> teuerstes Auto: In Niedersachsen für 165.482,31 Euro  </li>
            <li> günstigstes Auto: In Sachsen für 21.471,65 Euro </li>
        </ul>
</body>
</html>

# Kundenanalyse

<html>
<body>
<font size = "6">F7: Haben mehr Frauen oder mehr Männer unsere Autos gekauft?</font>
</body>
</html>

In [ ]:
geschlecht_vergleich = kunden_geo["Geschlecht"].value_counts()

labels = ["männlich", "weiblich"]
sizes = [700,400]
colors = ["lightblue","lightpink"]
plt.pie(sizes, labels = labels, colors = colors, autopct = "%1.1f%%", startangle = 90)
plt.axis("equal")
plt.title("Verteilung der verkauften Autos nach Geschlecht")
plt.show()


<html>
<body> 
<font size = "6">F8: Wie hoch ist das Durchschnittsalter unserer Kunden und unserer Besucher?</font>
        <h3>Durchschnittsalter der Kunden und Besucher</h3>
        <table border="1" class="dataframe" style="float: left;">
            <tr>
                <th>Kategorie</th>
                <th>Durchschnittsalter</th>
            </tr>
            <tr>
                <td>Kunden</td>
                <td>50,38 Jahre</td>
            </tr>
            <tr>
                <td>Besucher</td>
                <td>49,88 Jahre</td>
            </tr>
        </table>
</body>
</html>

<html>
<body>
<font size = "6">F9: Wie hoch ist das Durchschnittseinkommen unserer Kunden und unserer Besucher</font>
    <h3>Durchschnittseinkommen der Kunden und Besucher</h3>
        <table border="1" class="dataframe" style="float: left;">
            <tr>
                <th>Kategorie</th>
                <th>Durchschnittseinkommen</th>
            </tr>
            <tr>
                <td>Kunden</td>
                <td>58.850,91 Euro</td>
            </tr>
            <tr>
                <td>Besucher</td>
                <td>59.367,93 Euro</td>
            </tr>
        </table>
</body>
</html>

<html>
<body>
<font size = "6">13. Wie viele Kunden haben keinen Bankkredit aufgenommen, um das Auto zu kaufen?</font>
    <h3> Mögliches Risiko:</h3>
        <ul> 
            <li> 5 Kunden haben keinen Kredit aufgenommen  </li>
            <li> 99,55% unserer Kunden finanzieren sich ihr Auto mit einem Kredit</li>
        </ul>
</body>
</html>

# Einfluss der sozioökonomischen Faktoren

<html>
<body>
<font size = "6">F14: Welche sozioökonomischen Merkmale beeinflussen den Kaufpreis? </font><br>
    <table border="1" class="dataframe" style="float: left;">
        <tr>
                <th>Faktor</th>
                <th>Preiserhöhung pro 1 Einheit des Faktors</th>
            </tr>
            <tr>
                <td>Alter</td>
                <td>499,50 Euro</td>
            </tr>
            <tr>
                <td>Einkommen</td>
                <td>1,05 Euro</td>
            </tr>
            <tr>
                <td>Geschlecht (wenn männlich)</td>
                <td>2214,94 Euro</td>
            </tr>
    </table>
</body>
</html>

<html>
<body>
<font size = "8"> Ende der CEO Präsentation</font>
</body>
</html>

<html>
<body>
<font size = "8"> Präsentation für Vorgesetzte und Kollegen</font>
</body>
</html>

<font size = "6">10. Gibt es einen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher?</font>

In [ ]:
# H0:μ_kunden = μ_besucher
# H1:μ_kunden != μ_besucher
# α = 0.05

stats.ttest_ind(kunden_geo["Einkommen"], besucher_geo["Einkommen"], equal_var = True)


t_stat, p_val = stats.ttest_ind(kunden_geo["Einkommen"].dropna(), besucher_geo["Einkommen"].dropna(), equal_var=False)
t_stat, p_val

# Teststatistik = -0.7035, p-Wert = 0.4818
# Da p-Wert > α , kann die H0 nicht abgelehnt werden. Es liegen nicht genügende Anhaltspunkte vor, dass das Durchschnittseinkommen der Besucher und Kunden unterschiedlich ist

<font size = "6">11. Pearson-Korrelations-Koeffizient zwischen den Variablen Alter, Einkommen, Preis und Zeit</font>

In [ ]:
ausgewählte_spalten = ["Alter", "Einkommen", "Preis", "Zeit"]
korrelations_matrix = kunden_geo[ausgewählte_spalten].corr(method = 'pearson')

sns.heatmap(korrelations_matrix, annot = True, cmap = "coolwarm", linewidth = 1)
plt.title("Korrelationsmatrix")
plt.show()

<font size = "6">12. ist die Variable Zeit approximativ normalverteilt? (qq-Plot)>

In [ ]:
zeit = df_final["Zeit"]

# qq-Plot erstellen:
plt.figure(figsize = (8, 6))
stats.probplot(zeit, dist="norm", plot=plt)
plt.title('QQ-Plot der Variable "Zeit"')
plt.xlabel('Theoretische Quantile')
plt.ylabel('Beobachtete Quantile')
plt.show()
# da die Punkte entlang einer geraden Linie verlaufen kann davon ausgegangen werden, dass die Daten approvimativ normalverteilt sind

<font size = "6">13. Wie viele Kunden haben keinen Bankkredit aufgenommen, um das Auto zu kaufen? Also wie viele Kunden haben ein Jahreseinkommen, das über dem Autopreis liegt?</font>

In [ ]:
bedingung_einkommen_größer_preis = kunden_geo["Einkommen"] > kunden_geo["Preis"]
count_einkommen_größer_preis = bedingung_einkommen_größer_preis.sum()
count_einkommen_größer_preis
# 5 Kunden haben keinen Kredit aufnehmen müssen, um das Auto zu kaufen

<font size = "6">14. Welche sozioökonomischen Merkmale beeinflussen den Kaufpreis? Schätzen Sie eine Regression. </font>

In [ ]:
# unabhängige Variablen: Alter, Einkommen, Geschlecht
# abhängige Variable: Preis

y_kunden = kunden_geo["Preis"]

X_kunden = kunden_geo[['Alter', 'Einkommen', 'Geschlecht']]

X_kunden = sm.add_constant(X_kunden)  # Konstante für den Intercept
                
model_kunden = sm.OLS(y_kunden, X_kunden).fit()
model_kunden.summary()
# Regression: Preis = 692.5659 +499.5016*Alter +1.0485*Einkommen + 2214.9438*Geschlecht
# alle drei Faktoren haben einen statistischen Einfluss auf den Preis

<font size = "6"> 15. Prognose des Kaufpreises für 32-jährigen männlichen Kunden mit Einkommen von 30 000 Euro. Prognose des Kaufpreises für einen männlichen Kunden von 51 Jahren und Einkommen von 54 000 Euro.</font> 

In [ ]:
X_predict_kunden = pd.DataFrame({
    'Alter': [32, 51],
    'Einkommen': [30000, 54000],
    'Geschlecht': [1,1]
})

X_predict_kunden = sm.add_constant(X_predict_kunden, has_constant='add')

prediction_kunden = model_kunden.predict(X_predict_kunden)

prediction_kunden
# Das Modell schätzt den Kaufpreis eines 32-jährigen männlichen Kundens mit Einkommen von 30 000 Euro auf 50345.86 Euro
# Das Modell schätzt den Kaufpreis eines 51-jährigen männlichen Kundens mit Einkommen von 54 000 Euro auf 84999.83 Euro



<font size = "6"> 16. Welche Variable beeinflusst den Preis des Autos am
meisten? Vergleichbarkeit der Regressionskoeffizienten durch logistische Regression.</font>

In [ ]:
# unabhängige Variablen: Alter, Einkommen, Geschlecht
# abhängige Variable: Preis

# Standardisierung
X_stand = kunden_geo[['Alter', 'Einkommen', 'Geschlecht', "Preis"]]
Standard = X_stand.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

# Regression
regression = 'Preis ~ Alter + Einkommen + Geschlecht' 
result = smf.ols(regression, data=Standard).fit()

# Betas 
round(result.params, 6)

# Den meisten Einfluss auf den Autopreis hat das Einkommen mit einem Beta von rund 0,7.


<font size = "6">17. Regression, die die Wahrscheinlichkeit des Kaufs eines Autos ermittelt</font>

In [ ]:
# spalte einfügen, ob Kunde = 1 oder Besucher = 0
kunden_geo['Kauf'] = 1
besucher_geo['Kauf'] = 0

# Besucher unter Kunden anfügen
daten_alle = pd.concat([kunden_geo, besucher_geo])

# Überprüfung ob Käufer = 1 und Besucher = 0
daten_alle

In [ ]:
# Regression
y_lg = daten_alle["Kauf"]

X_lg = daten_alle[['Alter', 'Einkommen', 'Geschlecht', 'Zeit']]

X_lg = sm.add_constant(X_lg)

model_lg = sm.Logit(y_lg, X_lg).fit()

print(model_lg.summary())

<font size = "6">18. Wie hoch ist die Wahrscheinlichkeit, dass Person x,y unser Auto kauft?</font>

In [ ]:
# Kundendaten
Person_x = [1, 32, 30000, 1, 30]  # Kauf (1 = Kauf), Alter, Einkommen, Geschlecht (1 = männlich), Zeit 
Person_y = [1, 51, 54000, 1, 45] 

# Modell
params = model_lg.params

# Person X
log_1 = np.dot(params, Person_x)
wahrscheinlichkeit_person_x = 1 / (1 + np.exp(-log_1))

# Person Y 
log_2 = np.dot(params, Person_y)
wahrscheinlichkeit_person_y = 1 / (1 + np.exp(-log_2))

print(wahrscheinlichkeit_person_x)
print(wahrscheinlichkeit_person_y)

<font size = "6">19. Probleme bei der Zusammenführung des Datensatzes und Operationen zum Zusammenführen und Bereinigen der Daten</font>

In [ ]:
# 1.Umwandlung von Geschlecht in der Kunden-Datei von float in integer: kunden["Geschlecht"] = kunden["Geschlecht"].astype("int64")

# 2.Fehlende Daten in Kunden-Datei für Geschlecht:
# Herausgefunden mit der Operation kunden.isnull().sum(): zeigt die Werte mit "NaN", also wo keine Angaben sind
# Ermittlung des Modalwerts von Geschlecht: mit kunden["Geschlecht"].mode()
# Ersetzen der fehlenden Werte mit Modalwert: kunden.fillna(value = modalwert_kunden, axis = 0, inplace = True): setzt einen Wert für "NaN" ein, bei uns den Modalwert


# 3.Ausreißer in Kundendatei für Alter und Einkommen:
# Ausreißer beim Alter der Kunden identifizieren: kunden.loc[kunden['Ausreißer'] == True]: (2 Ausreißer: Personen im Alter von 100 und 120 Jahren werden kein Auto mehr kaufen)
# boxplot und describe-Tabelle zeigen die Ausreißer an: boxplot_alter = kunden.boxplot(column = 'Alter')
# kunden.describe().T
# Berechnung des Interquartilsabstandes: Q1, Q3 und IQR
# Q1_Alter = kunden['Alter'].quantile(0.25)
# Q3_Alter = kunden['Alter'].quantile(0.75)
# IQR_Alter = Q3_Alter - Q1_Alter

# Berechnung der Ausreißer-Grenzen:
# untere_grenze_Alter = Q1_Alter - 1.5 * IQR_Alter
# obere_grenze_Alter = Q3_Alter + 1.5 * IQR_Alter
# kunden['Ausreißer'] = (kunden['Alter'] < untere_grenze_Alter) | (kunden['Alter'] > obere_grenze_Alter)

# Ersetzen der Ausreißer in der Alters-Spalte durch Medianwert: alter_median = kunden['Alter'].median() und # kunden.replace([120, 100], alter_median, inplace=True)

# 4. Ausreißer-Werte beim Kunden-Einkommen identifiziert und mit dem Medianwert ersetzt (gleiches Prozedere wie bei Alter)

# 5. Bereinigung des Besucher-Datensatzes:
# Bereinigung der Kommaschreibweise und Umwandlung der dtypes von Einkommen, Zeit in Floats und Geschlecht in Integer
# Beispielhaft für Einkommen:
# besucher["Einkommen"] = besucher["Einkommen"].str.replace(",",".")
# besucher["Einkommen"] = pd.to_numeric(besucher["Einkommen"]) 
# besucher["Einkommen"] = besucher["Einkommen"].round(0).astype("float64")
# keine fehlenden Datenpunkte bei Besucher-Datei. Ausreißer entsprechen plausiblen Werten, daher wurden sie erhalten.

# 6.Umwandlung der geo-Datei von txt in csv: geo = pd.read_csv("geo.txt", sep = "\t")

# 7.Geo-Daten bereinigen: 
# Überprüfung aller einmaligen Werte: geo_niederlassung_uniques = geo["Niederlassung"].unique()

# Ersetzen der Werte, so dass jedes "Bundesland" nur einen Wert hat: 
# geo_cleaning = {
# "NRW" : "Nordrhein-Westfalen" ,
# "Berlin-Charlottenburg" : "Berlin" ,
# "Berlin-Mitte" : "Berlin" ,
# "BERLIN" : "Berlin",
# "Düsseldorf" : "Nordrhein-Westfalen"}

# geo["Niederlassung"] = geo["Niederlassung"].replace(geo_cleaning, regex = True)

# 8. Zusammenfügen der Kunden-Datei mit der Geo-Dateo: kunden_geo = pd.merge(kunden,geo, on = "KundeNr")
# Zusammefügen der Besucher-Datei mit der Geo-Datei 
# Zusammenfügen der Kunden-Geo und Besucher-Geo-Datei zu einer finalen Datei: df_final = pd.concat([kunden_geo, besucher_geo], axis = 0, ignore_index = True)
# CSV.Datei df_final exportieren: df_final.to_csv('df_final.csv', index=False)


<font size = "6">20. Vorschlaege für die Kundenabteilung</font>

- Einheitliche Daten: entweder Trennung durch Komma oder Punkt
- Vorgegebene Antwortmoeglichkeiten, um fehlende und fehlerhafte Angaben zu vermeiden
- Geo-Daten nach genauen Standorten zum Abgleich mit lokalem Einkommensniveau und lokalen Präferenzen (Stadt, Land)
- Weitere Informationen zu Kundenpräferenzen bezüglich Getriebe, Modell, Jahr, Kilometerstand (bei Gebrauchtwagen)
- Messung der Kundenzufriedenheit anhand einer Skala

In [ ]:
<font size = "8">Vielen Dank für Ihre Aufmerksamkeit!</font>